In [ ]:
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from glob import glob
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
base = MobileNetV2(input_shape = (128,128,3),include_top = False)

In [ ]:
x = base.output
x = Flatten()(x)
x = Dense(256,activation = 'relu')(x)
x = Dropout(0.3)(x)
x = Dense(1,activation='sigmoid')(x)

In [ ]:
model = Model(base.input,x)

In [ ]:
model.compile('adam','binary_crossentropy',['accuracy'])

In [ ]:
imgs = []
lbls = []

In [ ]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
for i in glob('/content/Face Mask Dataset/*/WithMask/*'):
  img = load_img(i,target_size = (128,128,3))
  img = img_to_array(img)
  imgs.append(img)
  lbls.append(1)
for i in glob('/content/Face Mask Dataset/*/WithoutMask/*'):
  img = load_img(i,target_size = (128,128,3))
  img = img_to_array(img)
  imgs.append(img)
  lbls.append(0)

In [ ]:
from sklearn.model_selection import train_test_split
imgs = np.array(imgs)
lbls = np.array(lbls)
trainx,testx,trainy,testy = train_test_split(imgs,lbls,stratify = lbls)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('chk',save_best_only=True)

In [ ]:
hist = model.fit(trainx,trainy,validation_data=(testx,testy),epochs = 10,batch_size = 128,callbacks=[mc])


In [ ]:
valx = []
valy = []
for i in glob('Validation/WithMask/*'):
  img = load_img(i,target_size=(128,128,3))
  img = img_to_array(img)
  valx.append(img)
  valy.append(1)
for i in glob('Validation/WithoutMask/*'):
  img = load_img(i,target_size=(128,128,3))
  img = img_to_array(img)
  valx.append(img)
  valy.append(0)
valx = np.array(valx)
valy = np.array(valy)

In [ ]:
model.evaluate(valx,valy,steps = 30)


In [ ]:
import matplotlib as mpl
mpl.rcParams.update({'font.size':16})
plt.figure(figsize = (10,6))
plt.plot(hist.history['accuracy'],label = 'Train')
plt.plot(hist.history['val_accuracy'],label = 'Test')
plt.title('Training v/s Testing accuracy with successive epochs')
plt.xlabel('Epochs')
#plt.yticks((0.94,1))
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
plt.figure(figsize = (10,6))
plt.plot(hist.history['loss'],label = 'Train')
plt.plot(hist.history['val_loss'],label = 'Test')
plt.title('Training v/s Testing Loss with successive epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
clf = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')


In [ ]:
img = cv2.imread('index.jpeg')
boxes = clf.detectMultiScale(img,1.01,5)
for box in boxes:
  (x,y,w,h) = box
  im = img[y:y+h,x:x+w]
  im = cv2.resize(im,(128,128))
  pred = model.predict(np.array([im]))
  if pred.round()==0:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),1)
    cv2.putText(img,'No Mask',(x,y),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),1)
  else:
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),1)
    cv2.putText(img,'Mask',(x,y),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),1)
plt.imshow(img)